# Document Classification
This tutorial will show how to perform document classification in Tribuo, using a variety of different methods to extract features from the text. We'll use the venerable [20-newsgroups dataset](http://qwone.com/~jason/20Newsgroups/) where the task is to predict what newsgroup a particular post is from, though this tutorial would be equally applicable to any document classification task (including tasks like sentiment analysis).

# Setup

You'll need a copy of the 20 newsgroups dataset, so first download and unpack it:

```
wget http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz
mkdir 20news
cd 20news
tar -zxf ../20news-bydate.tar.gz
```

This leaves you with two directories `20news-bydate-train` and `20news-bydate-test`, which contain the standard train and test split for this data.

20 newsgroups comes in a fairly standard format, the dataset is represented by a set of directories where the directory name is the class label, and the directory contains a collection of documents with one document in each file. Each file is a single Usenet post. For the purposes of this tutorial, we'll use the subject and body of the post as the input text for classification.

Here's an example:

```
$ ls 20news-bydate-train/
alt.atheism/               comp.sys.mac.hardware/  rec.motorcycles/     sci.electronics/         talk.politics.guns/
comp.graphics/             comp.windows.x/         rec.sport.baseball/  sci.med/                 talk.politics.mideast/
comp.os.ms-windows.misc/   misc.forsale/           rec.sport.hockey/    sci.space/               talk.politics.misc/
comp.sys.ibm.pc.hardware/  rec.autos/              sci.crypt/           soc.religion.christian/  talk.religion.misc/
$ ls 20news-bydate-train/comp.graphics/
37261  37949  38233  38270  38305  38344  38381  38417  38454  38489  38525  38562  38598  38633  38668  38703  38739
37913  37950  38234  38271  38306  38346  38382  38418  38455  38490  38526  38563  38599  38634  38669  38704  38740
37914  37951  38235  38272  38307  38347  38383  38420  38456  38491  38527  38564  38600  38635  38670  38705  38741
37915  37952  38236  38273  38308  38348  38384  38421  38457  38492  38528  38565  38601  38636  38671  38706  38742
...
```

As this is a pretty common format, Tribuo has a specific `DataSource` which can be used to read in this sort of data, `org.tribuo.data.text.DirectoryFileSource`.

We're going to use the classification experiments jar, along with the ONNX jar which provides support for loading in contextual word embedding models like [BERT](https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html).

In [ ]:
%jars ./tribuo-classification-experiments-4.1.0-SNAPSHOT-jar-with-dependencies.jar
%jars ./tribuo-interop-onnx-4.1.0-SNAPSHOT-jar-with-dependencies.jar

We'll also need a selection of imports from the `org.tribuo.data.text` package, along with the usual imports from `org.tribuo` and `org.tribuo.classification` we use when working with classification tasks. Weppor'll also load in the BERT support from the ONNX interop package. Tribuo's BERT support loads in models and tokenizers from [HuggingFace's Transformer](https://huggingface.co/transformers/) package, and can be easily extended to support non-BERT models.